In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [3]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
X=data.drop('EstimatedSalary',axis =1)
y=data['EstimatedSalary']

In [10]:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
with open ('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open ('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open ('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


ANN with regression problem statement 


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
model = Sequential([
    Dense(64 , activation='relu', input_shape= (X_train.shape[1],)),
    Dense(32 , activation ='relu'),
    Dense(1) ## output layer for regression
])

model.compile(optimizer ='adam', loss ='mean_absolute_error', metrics = ['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
import datetime

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

## setup the tensorboard
log_dir = "regression/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [18]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [20]:
history = model.fit(
    X_train, y_train,
    validation_data = (X_test,y_test),
    epochs =100,
    callbacks = [early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 7ms/step - loss: 100404.6016 - mae: 100404.6016 - val_loss: 98622.1562 - val_mae: 98622.1562
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 100012.9453 - mae: 100012.9453 - val_loss: 97803.0938 - val_mae: 97803.0938
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 98527.8125 - mae: 98527.8125 - val_loss: 95570.4922 - val_mae: 95570.4922
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 95441.9297 - mae: 95441.9297 - val_loss: 91627.0156 - val_mae: 91627.0156
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 90639.6562 - mae: 90639.6562 - val_loss: 86058.8438 - val_mae: 86058.8438
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 84366.0078 - mae: 84366.0078 - val_loss: 79356.6875 - val_mae: 79356.6875
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 77237.7656 - mae: 77237.7

In [21]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir regression/logs/fit

Reusing TensorBoard on port 6008 (pid 4656), started 0:00:05 ago. (Use '!kill 4656' to kill it.)

In [28]:
## Evaluate model on test data

test_loss, test_mae = model.evaluate(X_test,y_test)
print(f'Test MAE :{test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50250.8164 - mae: 50250.8164
Test MAE :50250.81640625


In [29]:
model.save('regression_model.h5')

d:\Data science\ANNclassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
